In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Here, I have import the dataseet from Tensorflow 

# Pre-processing

In [91]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Split data into training and testing sets
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_flat, y_train, test_size=0.2, random_state=42)


#ANN Implementation from scratch

In [92]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros((1, self.output_size))
# active
    def relu(self, z):
        return np.maximum(0, z)

    def softmax(self, z):
        exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)


    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        exp_scores = np.exp(self.z2)
        self.probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return self.probs
    def backward(self, X, y, learning_rate, reg_lambda):
    # def backward(self, X, y, learning_rate):
        num_examples = X.shape[0]
  # error
        delta3 = self.probs
        delta3[range(num_examples), y] -= 1
  #  lossFun hid
        dW2 = np.dot(self.a1.T, delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = np.dot(delta3, self.W2.T) * (self.a1 * (1 - self.a1))
  # loss inp
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)
        # Regularization terms
        dW2 += reg_lambda * self.W2
        dW1 += reg_lambda * self.W1
# update
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def fit(self, X, y, learning_rate=0.01, reg_lambda=0.01, epochs=1000, batch_size=32):
        for i in range(epochs):
            # Mini-batch SGD
            for j in range(0, X.shape[0], batch_size):
                X_batch = X[j:j+batch_size]
                y_batch = y[j:j+batch_size]
                self.forward(X_batch)
                self.backward(X_batch, y_batch, learning_rate, reg_lambda)

    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

#ANN using Keras

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train_split, y_train_split, epochs=10, validation_data=(x_val_split, y_val_split))


Epoch 1/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2880 - accuracy: 0.9177 - val_loss: 0.1683 - val_accuracy: 0.9525
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.1283 - accuracy: 0.9626 - val_loss: 0.1213 - val_accuracy: 0.9658
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0869 - accuracy: 0.9745 - val_loss: 0.0965 - val_accuracy: 0.9707
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0654 - accuracy: 0.9796 - val_loss: 0.0952 - val_accuracy: 0.9719
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0489 - accuracy: 0.9851 - val_loss: 0.0915 - val_accuracy: 0.9741
Epoch 6/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0397 - accuracy: 0.9879 - val_loss: 0.1012 - val_accuracy: 0.9714
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0302 - accuracy: 0.9904 - val_loss: 0.0890 - val_accuracy:

In [94]:
# Instantiate custom ANN
custom_ann = NeuralNetwork(input_size=784, hidden_size=128, output_size=10)

# Fit custom ANN
custom_ann.fit(x_train_split, y_train_split, learning_rate=0.01, reg_lambda=0.01, epochs=100, batch_size=32)

# Evaluate custom ANN
custom_ann_accuracy = np.mean(custom_ann.predict(x_test_flat) == y_test) * 100

# Evaluate Keras model
_, keras_accuracy = model.evaluate(x_test_flat, y_test)

print("Custom ANN Accuracy:", custom_ann_accuracy)


313/313 [==============================] - 1s 2ms/step - loss: 0.1095 - accuracy: 0.9680
Custom ANN Accuracy: 97.06


In [95]:
print("Keras Accuracy:", keras_accuracy*100)

Keras Accuracy: 96.79999947547913


## As we can see, Custom ANN Accuracy "97.06%" And Keras ANN Accuracy-> "96.73%". Therefore my scratch ANN code have more accuracy then keras !
